# Interactive Calibration with OpenPTV-Python
This notebook provides a step-by-step GUI for camera calibration using ipywidgets and OpenPTV-Python.

In [1]:
# Cell 2: Imports
import os
from pathlib import Path

import ipywidgets as widgets
from IPython.display import clear_output, display

# Ensure matplotlib plots are interactive in the notebook
%matplotlib widget

In [2]:
# Directory path chooser widget using ipywidgets
dir_chooser = widgets.Text(
    value=".",
    placeholder="Enter or paste directory path",
    description="Directory:",
    layout=widgets.Layout(width="80%"),
)

# Try to use FileChooser from ipyfilechooser if available
try:
    from ipyfilechooser import FileChooser

    file_chooser = FileChooser(
        path=".",
        select_dirs=True,
        show_only_dirs=True,
        title="Select Working Directory",
    )
    file_chooser._apply_path = lambda path: None  # Prevent auto-calling on init
    file_chooser_button = widgets.Button(
        description="Use Selected Folder", icon="check"
    )

    def on_file_chooser_select(b):
        dir_chooser.value = file_chooser.selected_path or file_chooser.path

    file_chooser_button.on_click(on_file_chooser_select)
    file_chooser_box = widgets.VBox([file_chooser, file_chooser_button])
except ImportError:
    file_chooser_box = widgets.HTML(
        "<i>ipyfilechooser not installed. File manager unavailable.</i>"
    )

dir_output = widgets.Output()


def on_dir_chooser_change(change):
    with dir_output:
        clear_output(wait=True)
        path = Path(change["new"])
        if not path.exists() or not path.is_dir():
            print(f"Invalid directory: {path}")
            return
        print(f"Selected working path: {path}\n")
        # List only directories and top 5 files (ignore hidden)
        dirs = [p for p in path.iterdir() if p.is_dir() and not p.name.startswith(".")]
        files = [
            p for p in path.iterdir() if p.is_file() and not p.name.startswith(".")
        ]
        print("Directories:")
        for d in dirs:
            print(f"  {d.name}/")
        print("\nTop 5 files:")
        for f in files[:5]:
            print(f"  {f.name}")
        # Assign valid path to working_dir
        global working_dir
        working_dir = str(path)


dir_chooser.observe(on_dir_chooser_change, names="value")

display(widgets.VBox([dir_chooser, file_chooser_box, dir_output]))

In [3]:
# working_dir = dir_chooser.value
print(working_dir)

/home/user/Documents/repos/test_cavity


In [7]:
# List all directories in working_dir that start with "parameters" but are not exactly "parameters"
param_dirs = [
    d
    for d in os.listdir(working_dir)
    if os.path.isdir(os.path.join(working_dir, d))
    and d.startswith("parameters")
    and d != "parameters"
]

# Extract the suffix after "parameters"
run_names = [d[len("parameters") :] for d in param_dirs]

# Create a dropdown widget for selecting the run
run_selection_dropdown = widgets.Dropdown(
    options=run_names,
    description="Select Run:",
    disabled=False,
    layout=widgets.Layout(width="50%"),
)

display(run_selection_dropdown)

Dropdown(description='Select Run:', layout=Layout(width='50%'), options=('Run3', 'Run1', 'Run2'), value='Run3'…

In [9]:
print(f"Selected run: {run_names[run_selection_dropdown.index]}")

Selected run: Run1


In [10]:
import shutil

# Source and destination directories
selected_run_folder = f"parameters{run_names[run_selection_dropdown.index]}"
src_dir = os.path.join(working_dir, selected_run_folder)
dst_dir = os.path.join(working_dir, "parameters")

# Ensure destination directory exists
os.makedirs(dst_dir, exist_ok=True)

# Copy all files from src_dir to dst_dir
for filename in os.listdir(src_dir):
    src_file = os.path.join(src_dir, filename)
    dst_file = os.path.join(dst_dir, filename)
    if os.path.isfile(src_file):
        shutil.copy2(src_file, dst_file)

print(f"Copied parameter files from {src_dir} to {dst_dir}")

Copied parameter files from /home/user/Documents/repos/test_cavity/parametersRun1 to /home/user/Documents/repos/test_cavity/parameters


In [35]:
# List all .par files in the dst_dir directory
par_files = [
    f
    for f in os.listdir(dst_dir)
    if f.endswith(".par") and os.path.isfile(os.path.join(dst_dir, f))
]
print("Parameter files in dst_dir:")
for f in par_files:
    print(f)

Parameter files in dst_dir:
multi_planes.par
criteria.par
track.par
detect_plate.par
sequence.par
shaking.par
man_ori.par
orient.par
targ_rec.par
examine.par
cal_ori.par
unsharp_mask.par
ptv.par
pft_version.par
dumbbell.par
sortgrid.par


In [ ]:
SORTGRID_EPS = 1  # Default value, adjust as needed


def read_sortgrid_par(filename) -> int:
    """Read the parameters for the sortgrid function from a file."""
    try:
        with open(filename, "r", encoding="utf-8") as fpp:
            eps = int(fpp.readline())
    except IOError:
        print(f"Can't open sortgrid parameter file: {filename} using default value")
        eps = SORTGRID_EPS

    return eps


# Usage example:
sortgrid_eps = read_sortgrid_par(os.path.join(dst_dir, "sortgrid.par"))
print(f"sortgrid_eps: {sortgrid_eps}")

In [ ]:
from openptv_python import parameters

# Mapping from .par file names to their corresponding parameter classes
par_class_map = {
    "ptv.par": parameters.ControlPar,
    "targets.par": parameters.TargetPar,
    "sequence.par": parameters.SequencePar,
    "criteria.par": parameters.VolumePar,
    "multi_planes.par": parameters.MultiPlanesPar,
    "track.par": parameters.TrackPar,
    "detect_plate.par": parameters.TargetPar,
    # "shaking.par": parameters.ShakingPar,
    # "man_ori.par": parameters.ManOriPar,
    "orient.par": parameters.OrientPar,
    "targ_rec.par": parameters.TargetPar,
    "examine.par": parameters.ExaminePar,
    "cal_ori.par": parameters.CalibrationPar,
    # "unsharp_mask.par": parameters.UnsharpMaskPar,
    "pft_version.par": parameters.PftVersionPar,
    # "dumbbell.par": parameters.DumbbellPar,
    # "sortgrid.par": parameters.SortgridPar,
}

par_objects = {}
for par_file in par_files:
    cls = par_class_map.get(par_file)
    file_path = os.path.join(dst_dir, par_file)
    if cls is not None and os.path.exists(file_path):
        try:
            par_objects[par_file] = cls(file_path)
        except Exception as e:
            print(f"Error reading {par_file}: {e}")
            par_objects[par_file] = None
    else:
        par_objects[par_file] = None

for fname, obj in par_objects.items():
    print(f"{fname}: {type(obj).__name__ if obj else 'None'}")


def read_all_parameters(param_dir):
    """
    Reads all relevant parameter files from the given directory and returns a dictionary
    of parameter objects. If a file is missing, the value is None.
    """  # noqa: D205, D401
    param_classes = {
        "cpar": ("ptv.par", parameters.ControlPar),
        "tpar": ("targets.par", parameters.TargetPar),
        "spar": ("sequence.par", parameters.SequencePar),
        "vpar": ("criteria.par", parameters.VolumePar),
        # Add more mappings here if needed
    }
    params = {}
    for key, (fname, cls) in param_classes.items():
        file_path = os.path.join(param_dir, fname)
        if os.path.exists(file_path):
            params[key] = cls(file_path)
        else:
            params[key] = None
    return params


# Usage example:
params = read_all_parameters(dst_dir)

for key, obj in params.items():
    print(f"{key} ({type(obj).__name__ if obj else 'None'}):")
    if obj is not None:
        for attr in dir(obj):
            if not attr.startswith("_") and not callable(getattr(obj, attr)):
                print(f"  {attr}: {getattr(obj, attr)}")
    else:
        print("  File not found.")
    print()

multi_planes.par: MultiPlanesPar
criteria.par: VolumePar
track.par: TrackPar
detect_plate.par: TargetPar
sequence.par: SequencePar
shaking.par: None
man_ori.par: None
orient.par: OrientPar
targ_rec.par: TargetPar
examine.par: ExaminePar
cal_ori.par: CalibrationPar
unsharp_mask.par: None
ptv.par: ControlPar
pft_version.par: PftVersionPar
dumbbell.par: None
sortgrid.par: None
cpar (ControlPar):
  all_cam_flag: 0
  cal_img_base_name: []
  chfield: 0
  hp_flag: 1
  img_base_name: []
  imx: 0
  imy: 0
  mm: nlay = 1, n1 = 1.0, n2 = [1.0], d = [0.0], n3 = 1.0
  num_cams: /home/user/Documents/repos/test_cavity/parameters/ptv.par
  pix_x: 0.0
  pix_y: 0.0
  tiff_flag: 1

tpar (None):
  File not found.

spar (SequencePar):
  first: 0
  img_base_name: /home/user/Documents/repos/test_cavity/parameters/sequence.par
  last: 0

vpar (VolumePar):
  cn: 0.0
  cnx: 0.0
  cny: 0.0
  corrmin: 0.0
  csumg: 0.0
  eps0: 0.0
  x_lay: /home/user/Documents/repos/test_cavity/parameters/criteria.par
  z_max_lay:

In [34]:
params["cpar"].get_num_cams()
params["cpar"].get_allCam_flag()

0